In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('hack_find').getOrCreate()
from pyspark.ml.clustering import KMeans

# Loads data.
dataset = spark.read.csv("hack_data.csv",header=True,inferSchema=True)
dataset.head()

ModuleNotFoundError: ignored

In [ ]:
dataset.describe().show()

+-------+-----------------------+------------------+------------------+-----------------+------------------+-----------+------------------+
|summary|Session_Connection_Time| Bytes Transferred|   Kali_Trace_Used|Servers_Corrupted|   Pages_Corrupted|   Location|  WPM_Typing_Speed|
+-------+-----------------------+------------------+------------------+-----------------+------------------+-----------+------------------+
|  count|                    334|               334|               334|              334|               334|        334|               334|
|   mean|     30.008982035928145| 607.2452694610777|0.5119760479041916|5.258502994011977|10.838323353293413|       NULL|57.342395209580864|
| stddev|     14.088200614636158|286.33593163576757|0.5006065264451406| 2.30190693339697|  3.06352633036022|       NULL| 13.41106336843464|
|    min|                    1.0|              10.0|                 0|              1.0|               6.0|Afghanistan|              40.0|
|    max|           

In [ ]:
df.columns

['A', 'B', 'C', 'D', 'Spoiled']

In [ ]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
feat_cols = ['Session_Connection_Time', 'Bytes Transferred', 'Kali_Trace_Used',
             'Servers_Corrupted', 'Pages_Corrupted','WPM_Typing_Speed']
vec_assembler = VectorAssembler(inputCols = feat_cols, outputCol='features')
final_data = vec_assembler.transform(dataset)
from pyspark.ml.feature import StandardScaler
scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures", withStd=True, withMean=False)
# Compute summary statistics by fitting the StandardScaler
scalerModel = scaler.fit(final_data)

In [ ]:
cluster_final_data = scalerModel.transform(final_data)

In [ ]:
from pyspark.ml.evaluation import ClusteringEvaluator
from pyspark.ml.clustering import KMeans

# Assuming cluster_final_data is your DataFrame with features

# Fit KMeans models
kmeans3 = KMeans(featuresCol='scaledFeatures', k=3)
kmeans2 = KMeans(featuresCol='scaledFeatures', k=2)
model_k3 = kmeans3.fit(cluster_final_data)
model_k2 = kmeans2.fit(cluster_final_data)

# Make predictions
predictions_k3 = model_k3.transform(cluster_final_data)
predictions_k2 = model_k2.transform(cluster_final_data)

# Evaluate clustering performance using ClusteringEvaluator
evaluator = ClusteringEvaluator(featuresCol='scaledFeatures', predictionCol='prediction', metricName='silhouette')

# Calculate silhouette score
silhouette_k3 = evaluator.evaluate(predictions_k3)
silhouette_k2 = evaluator.evaluate(predictions_k2)

# Print results
print("With K=3")
print("Silhouette Score = " + str(silhouette_k3))
print('--' * 30)
print("With K=2")
print("Silhouette Score = " + str(silhouette_k2))


With K=3
Silhouette Score = 0.7608455651454915
------------------------------------------------------------
With K=2
Silhouette Score = 0.817646009401248


In [ ]:
from pyspark.ml.evaluation import ClusteringEvaluator
from pyspark.ml.clustering import KMeans

# Assuming cluster_final_data is your DataFrame with features

for k in range(2, 9):
    # Fit KMeans model
    kmeans = KMeans(featuresCol='scaledFeatures', k=k)
    model = kmeans.fit(cluster_final_data)

    # Make predictions
    predictions = model.transform(cluster_final_data)

    # Evaluate clustering performance using ClusteringEvaluator
    evaluator = ClusteringEvaluator(featuresCol='scaledFeatures', predictionCol='prediction', metricName='silhouette')

    # Calculate silhouette score
    silhouette = evaluator.evaluate(predictions)

    # Print results
    print("With K={}".format(k))
    print("Silhouette Score = " + str(silhouette))
    print('--' * 30)


With K=2
Silhouette Score = 0.817646009401248
------------------------------------------------------------
With K=3
Silhouette Score = 0.7608455651454915
------------------------------------------------------------
With K=4
Silhouette Score = 0.7195901966635131
------------------------------------------------------------
With K=5
Silhouette Score = 0.6038978744953172
------------------------------------------------------------
With K=6
Silhouette Score = 0.5634378377200768
------------------------------------------------------------
With K=7
Silhouette Score = 0.5611091809595398
------------------------------------------------------------
With K=8
Silhouette Score = 0.48294292896878005
------------------------------------------------------------


In [ ]:
model_k3.transform(cluster_final_data).groupBy('prediction').count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|         1|   88|
|         2|   79|
|         0|  167|
+----------+-----+



In [ ]:
model_k2.transform(cluster_final_data).groupBy('prediction').count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|         1|  167|
|         0|  167|
+----------+-----+

